In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import tensorflow as tf

# Read CSV

In [2]:
fields = ['Date', 'Open','High','Low','Close','Volume','RSI','MACD','Ichimoku_Span_A','Ichimoku_Span_B','ATR',
         'BB_High_Indicator','BB_Low_Indicator']
MSFT_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/MSFT.csv", usecols=fields)
AAPL_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AAPL.csv", usecols=fields)
AMZN_df = pd.read_csv(r"/Users/fahad/project_repo/data/interim/AMZN.csv", usecols=fields)

MSFT_df.shape


(5414, 13)

In [3]:
#dict for the month endings which we will create a dataframe with the close prices from 
#apikey=539COEUG1C1IYXMF
MSFT = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=MSFT&outputsize=full&apikey=539COEUG1C1IYXMF")
MSFT = MSFT.json() 
AMZN = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=AMZN&outputsize=full&apikey=539COEUG1C1IYXMF")
AMZN = AMZN.json()
AAPL = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=AAPL&outputsize=full&apikey=539COEUG1C1IYXMF")
AAPL = AAPL.json()

In [4]:
#transform the json object into a pandas dataframe object and transpose
MSFT_df_monthly = pd.DataFrame(MSFT['Monthly Time Series'])
MSFT_df_monthly = MSFT_df_monthly.T
#transform the json object into a pandas dataframe object and transpose
AMZN_df_monthly = pd.DataFrame(AMZN['Monthly Time Series'])
AMZN_df_monthly = AMZN_df_monthly.T
#transform the json object into a pandas dataframe object and transpose
AAPL_df_monthly = pd.DataFrame(AAPL['Monthly Time Series'])
AAPL_df_monthly = AAPL_df_monthly.T

In [5]:
export_MSFT = MSFT_df_monthly.to_csv(r'MSFT_monthly.csv')
export_AMZN = AMZN_df_monthly.to_csv(r'AMZN_monthly.csv')
export_AAPL = AAPL_df_monthly.to_csv(r'AAPL_monthly.csv')

# format dataframe

In [6]:
MSFT_df_monthly.index.name = "Date"
MSFT_df_monthly.columns = ['Open', 'High','Low','Close','Volume']

#drop the last row of each dataframe so the entries match
#MSFT_df_monthly.drop(MSFT_df_monthly.tail(1).index,inplace=True)

AAPL_df_monthly.index.name="Date"
AAPL_df_monthly.columns =["Open","High","Low","Close","Volume"]
#AAPL_df_monthly.drop(AAPL_df_monthly.tail(1).index,inplace=True)

AMZN_df_monthly.index.name = "Date"
AMZN_df_monthly.columns = ["Open","High","Low","Close","Volume"]

# Labels

In [7]:
#empty dictionaries to store the labels
MSFT_labels = dict()

AAPL_labels = dict()

AMZN_labels = dict()

reversed_df_MSFT = MSFT_df_monthly.iloc[::-1]
reversed_df_AAPL= AAPL_df_monthly.iloc[::-1]
reversed_df_AMZN = AMZN_df_monthly.iloc[::-1]


def labelling(empty_dict, dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    
    prev_close =0
    counter = 0
    for index,cols in dataframe.iterrows():
        counter +=1
        current_close = cols['Close']
        if float(prev_close) <= float(current_close):
            #signals an increase in price between last month and next month
            empty_dict[index] = 1
        elif float(prev_close) >= float(current_close):
            empty_dict[index]=0

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

#ADD LABELS TO THE REVERESED DATAFRAMES USING THE LABELLING FUNCTION 
MSFT_labels = labelling(MSFT_labels,reversed_df_MSFT)
AAPL_labels = labelling(AAPL_labels,reversed_df_AAPL)
AMZN_labels = labelling(AMZN_labels,reversed_df_AMZN)

In [8]:
#Add the labels to the reversed DataFrames
reversed_df_MSFT['Labels'] = MSFT_labels.values()
reversed_df_AAPL['Labels'] = AAPL_labels.values()
reversed_df_AMZN['Labels']= AMZN_labels.values()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [9]:
#shouldn't have date 1998-02-27 in the final dataframe, because it is automatically assigned a label of 1
#do not know if the close is higher than the previous 30 days


# Set Index to Date 

In [10]:
# MSFT_df = MSFT_df.set_index('Date')
# AMZN_df = AMZN_df.set_index('Date')
# APPL_df = AAPL_df.set_index('Date')

In [11]:
def get_positive_days(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the REVERESED dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily. So for example will have 
    a dataframe of the month ending of february 2019-02-28:30days of January"""
    #dataframe_period_end is the monthly dataframe skips first entry and then begins creating key:value pairs from there
    # subperiod is the daily dataframe, which will have its rows form the value for the key
    empty_dict = dict()
    
    count = 0
    
    for end in dataframe_period_end.index:
        #print (end)
        #print ("\n")
        
        month_key = dataframe_period_end.index[count+1]
        #print ("month key:%s" % month_key)
        
        #dates which are smaller than the current month key, i.e get the previous dates
        prev = dataframe_subperiod['Date'] < month_key
        
        #dates which have a higher close price than open, essentially positive
        positive = dataframe_subperiod['Close'] > dataframe_subperiod['Open']
        
        last_index = dataframe_period_end.index[-1]
        
        if (month_key != last_index):
            #return only the first 15 entries that follow these two constraints
            temp_df = dataframe_subperiod[prev & positive].head(n)
            
            temp_df = temp_df.drop(['Date'], axis=1)
            
            empty_dict[month_key]= temp_df
            count +=1
        else:
            pass
        
       
    return empty_dict



In [12]:
def get_negative_days(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the REVERESED dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily. So for example will have 
    a dataframe of the month ending of february 2019-02-28:30days of January"""
    #dataframe_period_end is the monthly dataframe skips first entry and then begins creating key:value pairs from there
    # subperiod is the daily dataframe, which will have its rows form the value for the key
    empty_dict = dict()
    
    count = 0
    
    for end in dataframe_period_end.index:
        #print (end)
        #print ("\n")
        
        month_key = dataframe_period_end.index[count+1]
        #print ("month key:%s" % month_key)
        
        #dates which are smaller than the current month key, i.e get the previous dates
        prev = dataframe_subperiod['Date'] < month_key
        
        #dates which have a higher close price than open, essentially positive
        positive = dataframe_subperiod['Open'] > dataframe_subperiod['Close']
        
        last_index = dataframe_period_end.index[-1]
        
        if (month_key != last_index):
            #return only the first 15 entries that follow these two constraints
            temp_df = dataframe_subperiod[prev & positive].head(n)
            
            temp_df = temp_df.drop(['Date'], axis=1)
            
            empty_dict[month_key]= temp_df
            count +=1
        else:
            pass
        
       
    return empty_dict



# POSITIVE DICTIONARIES

In [13]:
MSFT_12days_positive = get_positive_days(reversed_df_MSFT, MSFT_df, 12)
AMZN_12days_positive = get_positive_days(reversed_df_AMZN, AMZN_df,12)
AAPL_12days_positive = get_positive_days(reversed_df_AAPL, AAPL_df,12)


# NEGATIVE DICTIONARIES

In [14]:
MSFT_12days_negative = get_negative_days(reversed_df_MSFT,MSFT_df,12)
AMZN_12days_negative = get_negative_days(reversed_df_AMZN, AMZN_df, 12)
AAPL_12days_negative = get_negative_days(reversed_df_AAPL, AAPL_df, 12)


# Function to create 4d Tensor for 2D CNN

In [15]:
def four_d_tensor(positive_dict, negative_dict):
    """ return a 4-d tensor of the positive and negative dates, for each key month date"""
    #list of lists that will have shape (756, 2, 12, 12)
    tensor_list = list()
    #dates to use as keys for the dicts 
    dates = list(positive_dict.keys())
    
    for date in dates:
        print (date)
        
#         #positive dataframe object corresponding to particular key will be 12x12 matrix

        positive_df = positive_dict[date]
    
#         #arr with shape 12 by 12

        positive_arr = np.array(positive_df)
    
        
#         #negative dataframe object 
        negative_df = negative_dict[date]
        
        
        
#         #negative array object
        negative_arr = np.array(negative_df)
        
#         #stack the two arrays on top of eachother so shape becomes 24 by 12
        df = np.vstack((positive_arr, negative_arr))
        
#         #3D tensor 
        three_tensor = np.array(np.vsplit(df, 2))
    
        #print (three_tensor.shape)
        
        tensor_list.append(three_tensor)
        
    
    tensor_list = np.array(tensor_list)
        
    return tensor_list
        
        
MSFT_4d = four_d_tensor(MSFT_12days_positive, MSFT_12days_negative)
AMZN_4d = four_d_tensor(AMZN_12days_positive, AMZN_12days_negative)
APPL_4d = four_d_tensor(AAPL_12days_positive, AAPL_12days_negative)


1998-03-31
1998-04-30
1998-05-29
1998-06-30
1998-07-31
1998-08-31
1998-09-30
1998-10-30
1998-11-30
1998-12-31
1999-01-29
1999-02-26
1999-03-31
1999-04-30
1999-05-28
1999-06-30
1999-07-30
1999-08-31
1999-09-30
1999-10-29
1999-11-30
1999-12-31
2000-01-31
2000-02-29
2000-03-31
2000-04-28
2000-05-31
2000-06-30
2000-07-31
2000-08-31
2000-09-29
2000-10-31
2000-11-30
2000-12-29
2001-01-31
2001-02-28
2001-03-30
2001-04-30
2001-05-31
2001-06-29
2001-07-31
2001-08-31
2001-09-28
2001-10-31
2001-11-30
2001-12-31
2002-01-31
2002-02-28
2002-03-28
2002-04-30
2002-05-31
2002-06-28
2002-07-31
2002-08-30
2002-09-30
2002-10-31
2002-11-29
2002-12-31
2003-01-31
2003-02-28
2003-03-31
2003-04-30
2003-05-30
2003-06-30
2003-07-31
2003-08-29
2003-09-30
2003-10-31
2003-11-28
2003-12-31
2004-01-30
2004-02-27
2004-03-31
2004-04-30
2004-05-28
2004-06-30
2004-07-30
2004-08-31
2004-09-30
2004-10-29
2004-11-30
2004-12-31
2005-01-31
2005-02-28
2005-03-31
2005-04-29
2005-05-31
2005-06-30
2005-07-29
2005-08-31
2005-09-30

In [16]:
MSFT_4d.shape

(256, 2, 12, 12)

In [17]:
AMZN_4d.shape

(256, 2, 12, 12)

In [18]:
APPL_4d.shape

(256, 2, 12, 12)